In [ ]:
import traitlets
from traitlets.config.configurable import SingletonConfigurable
import cv2
import numpy as np
import pyrealsense2 as rs
import threading
import tensorrt as trt
from tensorrt_model import TRTModel
from ssd_tensorrt import load_plugins, parse_boxes,TRT_INPUT_NAME, TRT_OUTPUT_NAME
import ctypes
import os
import ctypes
mean = 255.0 * np.array([0.5, 0.5, 0.5])
stdev = 255.0 * np.array([0.5, 0.5, 0.5])

def bgr8_to_ssd_input(camera_value):
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1)).astype(np.float32)
    x -= mean[:, None, None]
    x /= stdev[:, None, None]
    return x[None, ...]

In [ ]:
class Camera(SingletonConfigurable):
    
    #this changing of this value will be captured by traitlets
    color_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)

        #flag to control the thread
        self.thread_runnning_flag = False
        
        #start the RGBD sensor
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        #start capture the first color image
        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image

        #start capture the first depth image
        depth_frame = frames.get_depth_frame()           
        depth_image = np.asanyarray(depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        self.depth_value = depth_colormap   

In [ ]:
def _capture_frames(self):
        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            frames = self.pipeline.wait_for_frames() #receive data from RGBD sensor
            
            color_frame = frames.get_color_frame() #get the color image
            image = np.asanyarray(color_frame.get_data()) #convert color image to numpy array
            self.color_value = image #assign the numpy array image to the color_value variable 

            depth_frame = frames.get_depth_frame() #get the depth image           
            depth_image = np.asanyarray(depth_frame.get_data()) #convert depth data to numpy array
                
            #we only consider the central area of the vision sensor
            depth_image[:190,:]=0
            depth_image[290:,:]=0
            depth_image[:,:160]=0
            depth_image[:,480:]=0
            
            #For object avoidance, we don't consider the distance that are lower than 100mm or bigger than 1000mm
            depth_image[depth_image<100]=0
            depth_image[depth_image>1000]=0
            
            #If all of the values in the depth image is 0, the depth[depth!=0] command will fail
            #we set a specific value here to prevent this failure
            depth_image[0,0]=2000
            
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            if (depth_image[depth_image!=0].min()<800):
                self.warning_flag=2
            elif (depth_image[depth_image!=0].min() <875):
                cv2.putText(depth_colormap, 'warning!!!', (320,240), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                self.warning_flag=1

            else:
                self.warning_flag=0
            self.depth_value = depth_colormap               
    
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread       

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data

In [ ]:
class ObjectDetector(object):
    
    def __init__(self, engine_path, preprocess_fn=bgr8_to_ssd_input):
        logger = trt.Logger()
        trt.init_libnvinfer_plugins(logger, '')
        load_plugins()
        self.trt_model = TRTModel(engine_path, input_names=[TRT_INPUT_NAME],output_names=[TRT_OUTPUT_NAME, TRT_OUTPUT_NAME + '_1'])
        self.preprocess_fn = preprocess_fn
        
    def execute(self, *inputs):
        trt_outputs = self.trt_model(self.preprocess_fn(*inputs))
        return parse_boxes(trt_outputs)
    
    def __call__(self, *inputs):
        return self.execute(*inputs)
        
model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
import sys

#create widgets for the displaying of the image
display_color = widgets.Image(format='jpeg', width='45%') #determine the width of the color image
display_depth = widgets.Image(format='jpeg', width='45%')  #determine the width of the depth image
layout=widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth],layout=layout) #horizontal 
display(sidebyside) #display the widget

#callback function, invoked when traitlets detects the changing of the color image
def process(change):
    
    image = change['new'] #retrieve data from the input dict
    display_color.value = bgr8_to_jpeg(cv2.resize(image,(160,120)))
    display_depth.value = bgr8_to_jpeg(cv2.resize(camera.depth_value,(160,120)))


In [ ]:
import time
from RobotClass import Robot
import colorsys
#initialize the Robot class
robot = Robot()

image_widget = widgets.Image(format='jpeg', width=300, height=300)

upperyellowval = (30,102,107)
loweryellowval = (28,87,100)###############################################

uppervalhsv = (0,0,100)
lowervalhsv = (1,1,81)

width = 640
height = 480
label_widget = widgets.IntText(value=1, description='tracked label')

targetedcolour = [0,0,0]
firstloop = True 
def processing(change):
    image = change['new']
    human = []
    imgsized= cv2.resize(image,(300,300))
    # compute all detected objects
    detections = model(imgsized)
    
    
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]

    for det in matching_detections : 
        bbox = det['bbox']
         #assign detectbox coordinates:
        topleftx =int(width*bbox[0])
        toplefty = int(height*bbox[1])
        bottomrightx = int(width*bbox[2])
        bottomrighty = int(height*bbox[3])
        #detectbox = cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
        detectbox = cv2.rectangle(image, (topleftx, toplefty), (bottomrightx, bottomrighty), (255, 0, 0), 2)
        
       
        middle_x = (int(width*bbox[0]) + int(width*bbox[2]))/2
        middle_y = (int(height*bbox[1]) + int(height*bbox[3]))/2

  if camera.warning_flag == 0:
        if len(matching_detections)==0: #change this for if target 
            robot.right(0.4)
        elif len(matching_detections)>0:
            if (middle_x < 200):
                robot.left(0.4)
            elif(middle_x > 440):
                robot.right(0.4)
            else:
                robot.forward(0.3)
    elif(camera.warning_flag == 2):
        robot.backward(0.2)
    elif(camera.warning_flag == 1):
        robot.stop()
    
    
       
        
    image_widget.value = bgr8_to_jpeg(image)
    
#the camera.observe function will monitor the color_value variable. If this value changes, the excecute function will be excuted.
camera.observe(processing, names='color_value')

#create widgets for the displaying of the image
display_color = widgets.Image(format='jpeg', width='45%') #determine the width of the color image
display_depth = widgets.Image(format='jpeg', width='45%')  #determine the width of the depth image
layout=widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth],layout=layout) #horizontal 
display(sidebyside) #display the widget

#callback function, invoked when traitlets detects the changing of the color image
def process(change):
    
    image = change['new'] #retrieve data from the input dict
    display_color.value = bgr8_to_jpeg(cv2.resize(image,(160,120)))
    display_depth.value = bgr8_to_jpeg(cv2.resize(camera.depth_value,(160,120)))

processing({'new': camera.color_value})
#the camera.observe function will monitor the color_value variable. If this value changes, the processing function will be excuted.
camera.observe(process, names='color_value')

#stop the robot
robot.stop()